In [1]:
import os
from PIL import Image
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=pd.read_json("/Users/harshit/Downloads/image_features.json")

FileNotFoundError: File /Users/harshit/Downloads/image_features.json does not exist

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   path      10000 non-null  object
 1   features  10000 non-null  object
dtypes: object(2)
memory usage: 234.4+ KB


In [ ]:
c_df = pd.read_csv("/Users/harshit/Downloads/final_m_df.csv")

In [ ]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Category_encoded     10000 non-null  float64
 1   Subcategory_encoded  10000 non-null  float64
 2   Concept_encoded      10000 non-null  float64
 3   image_features       10000 non-null  object 
 4   tag_vector           10000 non-null  object 
 5   popularity_score     10000 non-null  float64
dtypes: float64(4), object(2)
memory usage: 468.9+ KB


In [ ]:
c_df.drop('image_features', inplace=True,axis=1)

In [ ]:
df.drop('path',inplace=True,axis=1)

In [ ]:
df

,features
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.2604446411, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...
9995,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9996,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9997,"[0.0, 1.5452661514000001, 0.0, 0.0, 0.0, 0.0, ..."
9998,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
features_array = np.array(df['features'].tolist())

In [ ]:
pca_features = PCA(n_components=1000)
features_pca_result = pca_features.fit_transform(features_array)

In [ ]:
for i in range(998):
    df[f'features_PCA{i+1}'] = features_pca_result[:, i]

In [ ]:
df1 = df.copy()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 1000 entries, features to features_PCA999
dtypes: float64(999), object(1)
memory usage: 76.4+ MB


In [ ]:
df1.drop('features',inplace=True,axis=1)

In [ ]:
import ast

# Convert 'tag_vector' elements from string to list of floats
c_df['tag_vector'] = c_df['tag_vector'].apply(lambda x: x if isinstance(x, list) else ast.literal_eval(x))

In [ ]:
pca = PCA(n_components=4)
pca_result = pca.fit_transform(c_df['tag_vector'].values.tolist())

# Create a new DataFrame containing the principal components
df_pca = pd.DataFrame(pca_result, columns=['tags_PC1', 'tags_PC2', 'tags_PC3', 'tags_PC4'])

In [ ]:
df_pca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tags_PC1  10000 non-null  float64
 1   tags_PC2  10000 non-null  float64
 2   tags_PC3  10000 non-null  float64
 3   tags_PC4  10000 non-null  float64
dtypes: float64(4)
memory usage: 312.6 KB


In [ ]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Category_encoded     10000 non-null  float64
 1   Subcategory_encoded  10000 non-null  float64
 2   Concept_encoded      10000 non-null  float64
 3   tag_vector           10000 non-null  object 
 4   popularity_score     10000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 390.8+ KB


In [ ]:
c_df.drop('tag_vector',inplace=True,axis=1)

In [ ]:
merged_df = pd.merge(c_df, df1, left_index=True, right_index=True, how='inner')
merged_df = pd.merge(merged_df, df_pca, left_index=True, right_index=True, how='inner')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 1007 entries, Category_encoded to tags_PC4
dtypes: float64(1007)
memory usage: 76.8 MB


In [ ]:
merged_df.head()

,Category_encoded,Subcategory_encoded,Concept_encoded,popularity_score,features_PCA1,features_PCA2,features_PCA3,features_PCA4,features_PCA5,features_PCA6,...,features_PCA994,features_PCA995,features_PCA996,features_PCA997,features_PCA998,features_PCA999,tags_PC1,tags_PC2,tags_PC3,tags_PC4
0,6.422156,6.515767,6.500651,11.18,21.693635,-17.767733,-66.077372,35.822643,16.598883,15.136580,...,-4.152699,0.585747,13.550130,7.301738,3.130817,2.689941,-0.071579,-0.003371,-0.010149,-0.013795
1,6.562402,6.603636,6.572583,6.74,-13.962764,-27.769074,1.440274,73.874523,-4.843766,88.011231,...,12.605338,-6.356803,6.026920,0.678395,-17.899320,6.511983,-0.048013,0.006811,-0.022045,0.072509
2,6.428840,6.468631,6.355811,5.45,7.633656,19.903929,27.318132,-28.678703,11.081023,11.714532,...,11.765264,4.564828,8.226883,-6.607529,-8.160454,9.603480,0.095578,0.186698,0.030619,0.018712
3,6.562402,6.603636,6.282670,7.05,-96.405011,48.497234,17.641654,4.975487,25.517619,-39.160556,...,-5.681020,-0.299498,5.374346,-4.240091,3.277401,-5.346504,0.003118,-0.039936,0.099476,-0.011788
4,6.562402,6.603636,6.282670,5.43,-102.097296,39.460716,31.280094,15.148222,24.529929,-27.140244,...,-8.945377,0.034102,0.230895,-3.800853,1.788044,5.337326,-0.046778,0.035949,0.030837,-0.021555


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# Splitting the data into features (X) and target variable (y)
X = merged_df.drop(columns=['popularity_score'])
y = merged_df['popularity_score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, ...)

In [ ]:
y_pred = xg_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)

Root Mean Squared Error: 2.595053385513782


In [ ]:
df.head()

,features,features_PCA1,features_PCA2,features_PCA3,features_PCA4,features_PCA5,features_PCA6,features_PCA7,features_PCA8,features_PCA9,...,features_PCA990,features_PCA991,features_PCA992,features_PCA993,features_PCA994,features_PCA995,features_PCA996,features_PCA997,features_PCA998,features_PCA999
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.2604446411, ...",21.693635,-17.767733,-66.077372,35.822643,16.598883,15.136580,0.410400,33.623031,-11.806736,...,5.911190,0.021956,1.743364,7.549202,-4.152699,0.585747,13.550130,7.301738,3.130817,2.689941
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-13.962764,-27.769074,1.440274,73.874523,-4.843766,88.011231,9.038059,29.852628,53.102603,...,-3.997741,7.033177,-0.397122,-1.671738,12.605338,-6.356803,6.026920,0.678395,-17.899320,6.511983
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",7.633656,19.903929,27.318132,-28.678703,11.081023,11.714532,45.198083,-35.959253,15.330220,...,-2.818395,8.938468,6.655533,6.598770,11.765264,4.564828,8.226883,-6.607529,-8.160454,9.603480
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-96.405011,48.497234,17.641654,4.975487,25.517619,-39.160556,7.243736,49.123334,-32.473589,...,-1.372694,0.216813,-1.829493,1.764209,-5.681020,-0.299498,5.374346,-4.240091,3.277401,-5.346504
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-102.097296,39.460716,31.280094,15.148222,24.529929,-27.140244,16.372911,41.747883,-45.050998,...,0.392454,-1.419967,1.200787,-0.812359,-8.945377,0.034102,0.230895,-3.800853,1.788044,5.337326


In [ ]:
df = df[['features']]

In [ ]:
df.head()

,features
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.2604446411, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
df = df.join(df['features'].apply(pd.Series))
df = df.drop('features', axis=1)

In [ ]:
df.info()